# Single Router - Standard Measurements

In [1]:
import sys
sys.path.append('..') # Search for custom module in the top level. 

# Import my custom modules.
from allens_quantum_package.functions import * 
from allens_quantum_package.operators import *

from qiskit import *
from qiskit.quantum_info import state_fidelity
from qiskit_ibm_runtime import QiskitRuntimeService

import scipy
import numpy
from numpy import set_printoptions, radians, ndarray, radians, array, sqrt

import itertools

In [2]:
# Set the floating point diplay precision to 3 decimal places, sufficient for our purposes.
set_printoptions(precision=3)

# Initialise the Qiskit runtime service. 
service = QiskitRuntimeService()

In [3]:
states = [
    (radians(-76), radians(-88)),
    (radians(93), radians(-135)),
    (radians(104), radians(-96))
]

In [4]:
def build_circuit(theta, phi) -> QuantumCircuit:
    circ = QuantumCircuit(3)

    circ.h(0)
    circ.u(theta, phi, 0, 1)
    circ.cswap(0, 1, 2)

    return circ

In [5]:
build_circuit(*states[0]).draw()

┌───┐             
q_0: ─────────┤ H ├───────────■─
     ┌────────┴───┴─────────┐ │ 
q_1: ┤ U(-1.3265,-1.5359,0) ├─X─
     └──────────────────────┘ │ 
q_2: ─────────────────────────X─

In [6]:
def add_basis_meas(circuit, qubit, basis='z') -> QuantumCircuit:
    circ = circuit.copy()
    circ.barrier()

    if basis == 'x':
        circ.h(qubit)
    elif basis == 'y':
        circ.sdg(qubit)
        circ.h(qubit)

    circ.measure_all()

    return circ


# Create named tuple object, for easy indexing, e.g. ".x", instead of "[0]"
TomographySet = namedtuple('TomographySet', ['x', 'y', 'z'])


def get_tomography_circuits(circuit, qubit) -> list[QuantumCircuit]:
    return [
        add_basis_meas(circuit, qubit, 'x'),
        add_basis_meas(circuit, qubit, 'y'),
        add_basis_meas(circuit, qubit, 'z')
    ]

In [7]:
get_tomography_circuits(build_circuit(*states[0]), 2)[0].draw()

┌───┐              ░       ░ ┌─┐      
   q_0: ─────────┤ H ├───────────■──░───────░─┤M├──────
        ┌────────┴───┴─────────┐ │  ░       ░ └╥┘┌─┐   
   q_1: ┤ U(-1.3265,-1.5359,0) ├─X──░───────░──╫─┤M├───
        └──────────────────────┘ │  ░ ┌───┐ ░  ║ └╥┘┌─┐
   q_2: ─────────────────────────X──░─┤ H ├─░──╫──╫─┤M├
                                    ░ └───┘ ░  ║  ║ └╥┘
meas: 3/═══════════════════════════════════════╩══╩══╩═
                                               0  1  2

In [8]:
def generate_tomography_sets(theta, phi) -> list[list[QuantumCircuit]]:
    return [ 
        *get_tomography_circuits(
            build_circuit(theta, phi), 1
        ),
        *get_tomography_circuits(
            build_circuit(theta, phi), 2
        )
    ]

In [9]:
ibm_brisbane = service.get_backend('ibm_brisbane')

In [10]:
circuits_to_send = list(itertools.chain.from_iterable([
        circuit for circuit in [
            itertools.chain.from_iterable(generate_tomography_sets(theta, phi) for theta, phi in states)
        ]
    ]
))

circuits_to_send = [transpile(circuit, ibm_brisbane) for circuit in circuits_to_send]

In [11]:
def get_all_physical_qubits(circuit_list: list[QuantumCircuit]) -> set[int]:
    all_indices = set()
    for circ in circuit_list:    
        measurement_indices = [instr[1][0]._index for instr in circ.data if instr[0].name == 'measure']
        all_indices = all_indices.union(measurement_indices)
    return all_indices


def get_mitigation_circuits(circuit_list: list[QuantumCircuit]) -> list[QuantumCircuit]:

    physical_qubits = get_all_physical_qubits(circuit_list)

    num_qubits = len(physical_qubits)
    all_0 = QuantumCircuit(127, num_qubits)
    all_0.measure(physical_qubits, range(num_qubits))

    all_1 = QuantumCircuit(127, num_qubits)
    all_1.x(physical_qubits)
    all_1.measure(physical_qubits, range(num_qubits))

    return [all_0, all_1]

In [12]:
circuits_to_send = get_mitigation_circuits(circuits_to_send) + circuits_to_send

In [13]:
hardware_job = ibm_brisbane.run(circuits=circuits_to_send)

In [14]:
print(f"Hardware job ID: {hardware_job.job_id()}")

Hardware job ID: cw6gprvggr6g0087cev0


---

In [15]:
results = hardware_job.result().get_counts()

In [16]:
def get_combined_xyz_counts_for_circuit(counts_list: list) -> tuple[dict]:

    output_qubit_1 = []
    for counts in counts_list[0:3]:
        counts_dict = {}

        counts_dict['0'] = sum(count for bit_string, count in counts.items() if bit_string[2] == '0' and bit_string[1] == '0')
        counts_dict['1'] = sum(count for bit_string, count in counts.items() if bit_string[2] == '0' and bit_string[1] == '1')

        output_qubit_1.append(counts_dict)

    output_qubit_2 = []
    for counts in counts_list[3:6]:
        counts_dict = {}
        
        counts_dict['0'] = sum(count for bit_string, count in counts.items() if bit_string[2] == '1' and bit_string[0] == '0')
        counts_dict['1'] = sum(count for bit_string, count in counts.items() if bit_string[2] == '1' and bit_string[0] == '1')       

        output_qubit_2.append(counts_dict)

    output = (add_dicts(dict_1, dict_2) for dict_1, dict_2 in zip(output_qubit_1, output_qubit_2))   
    
    return tuple(output)


def print_unmitigated_fidelities_combined(counts, theta, phi):

    qubit = density_op_from_counts_dict(*get_combined_xyz_counts_for_circuit(counts))

    print(get_combined_xyz_counts_for_circuit(counts))
    print(qubit)

    psi = gen_qubit(theta, phi)

    fidelity_q = state_fidelity(qubit, psi)

    print(f'State fidelity from combined counts: {fidelity_q}\n')

In [17]:
print_unmitigated_fidelities_combined(results[2:8], *states[0])
print_unmitigated_fidelities_combined(results[8:14], *states[1])
print_unmitigated_fidelities_combined(results[14:20], *states[2])

({'0': 2409, '1': 1582}, {'0': 3670, '1': 386}, {'0': 2492, '1': 1495})
[[0.625+0.j    0.104-0.405j]
 [0.104+0.405j 0.375+0.j   ]]
State fidelity from combined counts: 0.9193071661235508

({'0': 420, '1': 3585}, {'0': 1291, '1': 2753}, {'0': 1795, '1': 2214})
[[ 0.448+0.j    -0.395+0.181j]
 [-0.395-0.181j  0.552+0.j   ]]
State fidelity from combined counts: 0.9093945037404041

({'0': 1206, '1': 2777}, {'0': 467, '1': 3498}, {'0': 1416, '1': 2540})
[[ 0.358+0.j    -0.197+0.382j]
 [-0.197-0.382j  0.642+0.j   ]]
State fidelity from combined counts: 0.9232043554687126



In [18]:
# Gets the map from a physical qubit to a classical bit for the mitigation calibration matrices
def get_qubit_to_clbit_mappings_for_mitigation(circuit: QuantumCircuit) -> dict[int, int]:
    return dict(
        ((instr[1][0]._index, instr[2][0]._index) for instr in circuit.data if instr[0].name == 'measure')
    )

# Gets the map from physical qubit to mitigation matrix
def get_assignment_matrices(mappings: dict, counts_0: dict, counts_1: dict) -> dict[int, ndarray]:

    output = {}

    for physical_qubit, classical_bit in mappings.items():
        
        # Determine zero state for assignment matrix
        result_0 = sum([count for bit_string, count in counts_0.items() if bit_string[classical_bit] == '0'])
        result_1 = sum([count for bit_string, count in counts_0.items() if bit_string[classical_bit] == '1'])
        
        # Calculate zero ket
        zero_ket = array([[result_0], 
                          [result_1]]) / (result_0 + result_1)
        
        # Determine one state for assignment matrix
        result_0 = sum([count for bit_string, count in counts_1.items() if bit_string[classical_bit] == '0'])
        result_1 = sum([count for bit_string, count in counts_1.items() if bit_string[classical_bit] == '1'])
        
        # Calculate zero ket
        one_ket = array([[result_0], 
                         [result_1]]) / (result_0 + result_1)
        
        assignment_matrix = numpy.concatenate([zero_ket, one_ket], axis=1)

        output[physical_qubit] = assignment_matrix
    
    return output

# Get the list of physical qubits in the same order as the classical bits 
def get_qubits_in_clbit_order(circuit: QuantumCircuit) -> list[int]:
    
    cl_bit_to_qbit_map = dict((instr[2][0]._index, instr[1][0]._index) for instr in circuit.data if instr[0].name == 'measure')

    return [*cl_bit_to_qbit_map.values()]
    

# Get the tensor of all mitigation matrices in the correct order for each circuit
def get_mitigation_matrix(circuit: QuantumCircuit, assignment_mappings: dict) -> ndarray:
    ordered_physical_qubits = get_qubits_in_clbit_order(circuit)

    ordered_physical_qubits.reverse()

    assignment_matrices = [assignment_mappings[qubit] for qubit in ordered_physical_qubits]

    return tens(*(scipy.linalg.inv(mat) for mat in assignment_matrices))


def get_bit_strings(count: int) -> list[str]:
    return [''.join(bits) for bits in itertools.product(['0', '1'], repeat=count)]


def get_corrected_counts(circuit: QuantumCircuit, counts_for_mitigation: dict, assignment_mappings: dict, num_qubits: int) -> dict[str, int]:
    bit_strings = get_bit_strings(num_qubits)

    counts_list = []
    for bit_string in bit_strings:
        counts_list.append(counts_for_mitigation[bit_string] if bit_string in counts_for_mitigation else 0)
    
    # Get vector of counts
    counts_vector = numpy.concatenate(
        array([[count for count in counts_list]]),
        axis=0
    )

    mitigation_matrix = get_mitigation_matrix(circuit, assignment_mappings)
    
    # Multiply by mitigation matirx
    corrected_vector = mitigation_matrix @ counts_vector
    corrected_vector = corrected_vector.astype(int)
    
    output = {}
    for idx, bit_string in zip(range(2**num_qubits), bit_strings):
        output[bit_string] = int(corrected_vector[idx])
    
    return output

In [19]:
# Get list of qubit to clbit mappings for the mitigation circuits.
mitigation_mappings = get_qubit_to_clbit_mappings_for_mitigation(circuits_to_send[0])

# Calculate assignment matrices for each qubit. 
assignment_matrices = get_assignment_matrices(mitigation_mappings, results[0], results[1])

In [20]:
def print_mitigated_fidelities_combined(circuits, counts, assignment_matrices, num_qubits, theta, phi):

    mitigated_counts = []

    for circuit, count in zip(circuits, counts):
        mitigated_counts.append(get_corrected_counts(circuit, count, assignment_matrices, num_qubits))

    qubit = density_op_from_counts_dict(*get_combined_xyz_counts_for_circuit(mitigated_counts))
    
    print(get_combined_xyz_counts_for_circuit(mitigated_counts))
    print(qubit)

    psi = gen_qubit(theta, phi)

    fidelity_q = state_fidelity(qubit, psi)

    print(f'Mitigated state fidelity from combined counts: {fidelity_q}\n')

In [21]:
print_mitigated_fidelities_combined(circuits_to_send[2:8], results[2:8], assignment_matrices, 3, *states[0])
print_mitigated_fidelities_combined(circuits_to_send[8:14], results[8:14], assignment_matrices, 3, *states[1])
print_mitigated_fidelities_combined(circuits_to_send[14:20], results[14:20], assignment_matrices, 3, *states[2])

({'0': 2415, '1': 1572}, {'0': 3714, '1': 339}, {'0': 2476, '1': 1507})
[[0.622+0.j    0.106-0.416j]
 [0.106+0.416j 0.378+0.j   ]]
Mitigated state fidelity from combined counts: 0.9295924607000129

({'0': 363, '1': 3638}, {'0': 1261, '1': 2780}, {'0': 1758, '1': 2248})
[[ 0.439+0.j    -0.409+0.188j]
 [-0.409-0.188j  0.561+0.j   ]]
Mitigated state fidelity from combined counts: 0.924921196874563

({'0': 1175, '1': 2805}, {'0': 413, '1': 3548}, {'0': 1367, '1': 2584})
[[ 0.346+0.j    -0.205+0.396j]
 [-0.205-0.396j  0.654+0.j   ]]
Mitigated state fidelity from combined counts: 0.9399026288028747

